$\#\#\#\#\#\#\#$ Libraries $\#\#\#\#\#\#\#$

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import os
import requests

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup



$\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#$

# Nivell 1

## Exercici 1: Realitza web scraping d'una pàgina de la [borsa de Madrid](https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

This exercise asks us to do scraping using two different libraries: BeautifulSoup and Selenium. We will begin with BS and then with Selenium we will do the same.

Let's begin with fetching the url,

In [4]:
url = 'https://www.bolsamadrid.es/'

**WebScraping using BeautifulSoup**

> I have been using the tutorial: https://brightdata.com/blog/how-tos/how-to-use-beautiful-soup-for-web-scraping-with-python to undertand how to do webscraping with this library

In [9]:
# request url and take a look at the content:
req = requests.get(url)
#print(req.content)

In [49]:
# let's get it structured with beautiful soup!
soup = BeautifulSoup(req.content, 'html5lib')
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml"><head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Fri, 08 Apr 2022 16:39:06 GMT" data-idioma="esp"><meta content="IE=11" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="Copyright © BME 2022" id="ctl00_copyright" name="copyright"/><title>
	Bolsa de Madrid
</title><link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" title="Bolsa de Madrid: Regulación: Circulares e In

On the webpage we can see that on the menu on the left, there is a link were we can get all shares of the Market given a day. Thus this is the table we want to download.

We locate the link to the webpage with find_all

In [50]:
soup.find_all('a', string='Acciones')

[<a href="/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000" target="_self">Acciones</a>,
 <a href="/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000" target="_self">Acciones</a>]

In [51]:
# we add the link to the webpage
hlinks = []
for link in soup.find_all('a', string='Acciones'):
    hlinks.append(link.get('href'))

hlinks

['/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000',
 '/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000']

In [52]:
newpage = url + hlinks[0] # we only need one, they are the same
newpage

'https://www.bolsamadrid.es//esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [59]:
# download the table from the webpage:
req2 = requests.get(newpage)
web2 = BeautifulSoup(req2.content, 'html5lib')
print(web2)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml"><head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Fri, 08 Apr 2022 17:16:55 GMT" data-idioma="esp"><meta content="IE=11" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="Copyright © BME 2022" id="ctl00_copyright" name="copyright"/><title>
	Bolsa de Madrid - Precios de la sesión
</title><link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" title="Bolsa de Madrid: Regu

Now we need to extract the content from "< table cellpadding="3" cellspacing="0" class="TblPort" id="ctl00_Contenido_tblAcciones" width="100%"> " since there is where the data is.

In [63]:
table0 = web2.find(id="ctl00_Contenido_tblAcciones")
print(table0)

<table cellpadding="3" cellspacing="0" class="TblPort" id="ctl00_Contenido_tblAcciones" width="100%">
		<tbody><tr align="center">
			<th scope="col">Nombre</th><th scope="col">Últ.</th><th scope="col">% Dif.</th><th scope="col">Máx.</th><th scope="col">Mín.</th><th scope="col">Volumen</th><th scope="col">Efectivo (miles €)</th><th scope="col">Fecha</th><th class="Ult" scope="col">Hora</th>
		</tr><tr align="right">
			<td align="left" class="DifFlSb"><a href="/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0125220311">ACCIONA</a></td><td>191,1000</td><td class="DifClSb">2,19</td><td>192,1000</td><td>187,3000</td><td>69.379</td><td>13.189,85</td><td align="center">08/04/2022</td><td align="center" class="Ult">17:00:38</td>
		</tr><tr align="right">
			<td align="left" class="DifFlSb"><a href="/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0132105018">ACERINOX</a></td><td>9,4900</td><td class="DifClSb">1,17</td><td>9,6820</td><td>9,4320</td><td>680.041</td><td>6.502,01</td><td align="center">08/04

In [65]:
# now let get the columns names with the values between the ths, we put them into a list.
print(table0.find_all('th'))

cols = []
for n in table0.find_all('th'):
    cols.append(n.get_text())

cols

[<th scope="col">Nombre</th>, <th scope="col">Últ.</th>, <th scope="col">% Dif.</th>, <th scope="col">Máx.</th>, <th scope="col">Mín.</th>, <th scope="col">Volumen</th>, <th scope="col">Efectivo (miles €)</th>, <th scope="col">Fecha</th>, <th class="Ult" scope="col">Hora</th>]


['Nombre',
 'Últ.',
 '% Dif.',
 'Máx.',
 'Mín.',
 'Volumen',
 'Efectivo (miles €)',
 'Fecha',
 'Hora']

In [71]:
# We see that the values between tds are the observations of each column, we put them into a list:
elemts = []

for n in table0.find_all('td'):
    elemts.append(n.get_text())

elemts

['ACCIONA',
 '191,1000',
 '2,19',
 '192,1000',
 '187,3000',
 '69.379',
 '13.189,85',
 '08/04/2022',
 '17:00:38',
 'ACERINOX',
 '9,4900',
 '1,17',
 '9,6820',
 '9,4320',
 '680.041',
 '6.502,01',
 '08/04/2022',
 '17:01:22',
 'ACS',
 '24,0600',
 '2,17',
 '24,0600',
 '23,7300',
 '343.759',
 '8.213,91',
 '08/04/2022',
 '17:00:33',
 'AENA',
 '145,2000',
 '1,72',
 '145,2500',
 '142,8000',
 '524.961',
 '75.865,05',
 '08/04/2022',
 '17:00:11',
 'ALMIRALL',
 '12,0900',
 '0,17',
 '12,1900',
 '11,7900',
 '307.026',
 '3.692,84',
 '08/04/2022',
 '17:00:55',
 'AMADEUS',
 '56,5400',
 '0,18',
 '57,1600',
 '55,9600',
 '496.305',
 '28.162,72',
 '08/04/2022',
 '17:01:09',
 'ARCELORMIT.',
 '28,2600',
 '2,00',
 '28,7800',
 '28,0000',
 '309.861',
 '8.808,69',
 '08/04/2022',
 '17:00:11',
 'B.SANTANDER',
 '3,1240',
 '2,78',
 '3,1470',
 '3,0770',
 '25.286.093',
 '78.888,87',
 '08/04/2022',
 '17:01:22',
 'BA.SABADELL',
 '0,7350',
 '3,23',
 '0,7464',
 '0,7244',
 '39.381.708',
 '28.928,46',
 '08/04/2022',
 '17:00:3

In [88]:
# we need to restructure the list above into an array with 9 columns:
print('lenght columns', len(cols))
print('lenght elements', len(elemts))
print(len(elemts)/len(cols))

# we will do it with the code I wrote in sprint2 task1
grup = list()
numx = len(cols)
for x in range(0,len(elemts),numx):
    grup.append(elemts[x:x+numx])

#grup

lenght columns 9
lenght elements 315
35.0


In [90]:
# now we can create the dataframe
df = pd.DataFrame(grup)
df.columns = cols
df

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"191,1000","2,19","192,1000","187,3000",69.379,"13.189,85",08/04/2022,17:00:38
1,ACERINOX,"9,4900","1,17","9,6820","9,4320",680.041,"6.502,01",08/04/2022,17:01:22
2,ACS,"24,0600","2,17","24,0600","23,7300",343.759,"8.213,91",08/04/2022,17:00:33
3,AENA,"145,2000","1,72","145,2500","142,8000",524.961,"75.865,05",08/04/2022,17:00:11
4,ALMIRALL,"12,0900","0,17","12,1900","11,7900",307.026,"3.692,84",08/04/2022,17:00:55
5,AMADEUS,"56,5400","0,18","57,1600","55,9600",496.305,"28.162,72",08/04/2022,17:01:09
6,ARCELORMIT.,"28,2600","2,00","28,7800","28,0000",309.861,"8.808,69",08/04/2022,17:00:11
7,B.SANTANDER,"3,1240","2,78","3,1470","3,0770",25.286.093,"78.888,87",08/04/2022,17:01:22
8,BA.SABADELL,"0,7350","3,23","0,7464","0,7244",39.381.708,"28.928,46",08/04/2022,17:00:31
9,BANKINTER,"5,4260","3,20","5,4740","5,2920",2.035.936,"10.996,78",08/04/2022,17:00:49


In [ ]:
df.to_csv('MarketsBS.csv')

**WebScraping using Selenium**

To use Selenium, we follow this [tutorial](https://realpython.com/modern-web-automation-with-python-and-selenium/), and before anything else we install the geckodriver (instead of using the one from the tutorial, for compatibility its necessary to go to the repository and download version 0.31), with Selenium.

In [34]:
# set up Selenium
opts = Options()
opts.headless = True
assert opts.headless
# set up the browser:
browser = Firefox(options=opts)
url = 'https://www.bolsamadrid.es'
browser.get(url)

Now, we want to download the Markets as before, thus we do a search for the link, "Accciones" and click on it.

> From stack overflow a user posted a mini list of the different attributes forthe fucntion By. :
    >    CLASS_NAME, CSS_SELECTOR, ID, LINK_TEXT, NAME, PARTIAL_LINK_TEXT, TAG_NAME, XPATH

In [41]:
accions = browser.find_elements(By.LINK_TEXT, "Acciones")
print('lenght: ', len(accions))
print(accions[0].text)
accions[0].click()

lenght:  2
Acciones


In [48]:
browser.current_url

'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

Now, we are inside the webpage: https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000,
Let's download the table of the Markets.

To do so, we need to tap into the same id as before: id="ctl00_Contenido_tblAcciones"


In [52]:
table = browser.find_element(By.ID, 'ctl00_Contenido_tblAcciones')
print(table)
table.text

<selenium.webdriver.remote.webelement.WebElement (session="0a6a13b1-5e9e-47e9-b5f8-7f986fb36173", element="c4a617c4-e859-42b3-b460-13f1bc580055")>


'Nombre Últ. % Dif. Máx. Mín. Volumen Efectivo (miles €) Fecha Hora\nACCIONA 196,0000 -0,25 196,6000 192,7000 51.357 9.985,06 20/04/2022 14:05:01\nACERINOX 10,0700 1,45 10,1500 9,9300 549.250 5.513,00 20/04/2022 14:03:32\nACS 24,8300 0,36 24,9900 24,8100 195.254 4.856,16 20/04/2022 14:03:08\nAENA 149,7500 -0,27 150,5000 148,5500 37.897 5.663,36 20/04/2022 14:05:10\nALMIRALL 12,2900 -0,49 12,3600 12,1900 100.272 1.230,18 20/04/2022 14:04:21\nAMADEUS 57,8200 1,72 57,8800 56,5600 154.472 8.855,53 20/04/2022 14:05:02\nARCELORMIT. 30,4550 0,15 30,9350 30,2850 183.131 5.618,70 20/04/2022 14:03:50\nB.SANTANDER 3,2390 2,44 3,2495 3,1815 102.163.399 327.516,35 20/04/2022 14:05:04\nBA.SABADELL 0,7772 4,94 0,7800 0,7442 20.820.551 15.985,94 20/04/2022 14:05:13\nBANKINTER 5,4140 2,46 5,4480 5,3100 1.220.592 6.584,03 20/04/2022 14:03:07\nBBVA 5,1930 2,18 5,2020 5,1040 6.328.711 32.654,70 20/04/2022 14:05:04\nCAIXABANK 3,1280 3,06 3,1340 3,0510 4.810.027 14.949,37 20/04/2022 14:05:15\nCELLNEX 45,850

As before, we name the columns and map the table, remember that the column names are within "th" and the values between "td".

In [53]:
colus = []
for i in table.find_elements(By.TAG_NAME, 'th'):
    colus.append(i.text)
    
colus

['Nombre',
 'Últ.',
 '% Dif.',
 'Máx.',
 'Mín.',
 'Volumen',
 'Efectivo (miles €)',
 'Fecha',
 'Hora']

In [54]:
# now we get the Market values:
vals = []
for i in table.find_elements(By.TAG_NAME, 'td'):
    vals.append(i.text)
    
vals

['ACCIONA',
 '196,0000',
 '-0,25',
 '196,6000',
 '192,7000',
 '51.357',
 '9.985,06',
 '20/04/2022',
 '14:05:01',
 'ACERINOX',
 '10,0700',
 '1,45',
 '10,1500',
 '9,9300',
 '549.250',
 '5.513,00',
 '20/04/2022',
 '14:03:32',
 'ACS',
 '24,8300',
 '0,36',
 '24,9900',
 '24,8100',
 '195.254',
 '4.856,16',
 '20/04/2022',
 '14:03:08',
 'AENA',
 '149,7500',
 '-0,27',
 '150,5000',
 '148,5500',
 '37.897',
 '5.663,36',
 '20/04/2022',
 '14:05:10',
 'ALMIRALL',
 '12,2900',
 '-0,49',
 '12,3600',
 '12,1900',
 '100.272',
 '1.230,18',
 '20/04/2022',
 '14:04:21',
 'AMADEUS',
 '57,8200',
 '1,72',
 '57,8800',
 '56,5600',
 '154.472',
 '8.855,53',
 '20/04/2022',
 '14:05:02',
 'ARCELORMIT.',
 '30,4550',
 '0,15',
 '30,9350',
 '30,2850',
 '183.131',
 '5.618,70',
 '20/04/2022',
 '14:03:50',
 'B.SANTANDER',
 '3,2390',
 '2,44',
 '3,2495',
 '3,1815',
 '102.163.399',
 '327.516,35',
 '20/04/2022',
 '14:05:04',
 'BA.SABADELL',
 '0,7772',
 '4,94',
 '0,7800',
 '0,7442',
 '20.820.551',
 '15.985,94',
 '20/04/2022',
 '14:0

In [58]:
# we need to restructure the list above into an array with 9 columns:
print('lenght columns', len(colus))
print('lenght elements', len(vals))
print(len(vals)/len(colus))

# we will do it with the code I wrote in sprint2 task1
tb1 = list()
nux = len(colus)
for x in range(0,len(vals),nux):
    tb1.append(vals[x:x+nux])

#tb1

lenght columns 9
lenght elements 315
35.0


In [59]:
df2 = pd.DataFrame(tb1)
df2.columns = colus
df2

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"196,0000","-0,25","196,6000","192,7000",51.357,"9.985,06",20/04/2022,14:05:01
1,ACERINOX,"10,0700","1,45","10,1500","9,9300",549.250,"5.513,00",20/04/2022,14:03:32
2,ACS,"24,8300","0,36","24,9900","24,8100",195.254,"4.856,16",20/04/2022,14:03:08
3,AENA,"149,7500","-0,27","150,5000","148,5500",37.897,"5.663,36",20/04/2022,14:05:10
4,ALMIRALL,"12,2900","-0,49","12,3600","12,1900",100.272,"1.230,18",20/04/2022,14:04:21
5,AMADEUS,"57,8200","1,72","57,8800","56,5600",154.472,"8.855,53",20/04/2022,14:05:02
6,ARCELORMIT.,"30,4550","0,15","30,9350","30,2850",183.131,"5.618,70",20/04/2022,14:03:50
7,B.SANTANDER,"3,2390","2,44","3,2495","3,1815",102.163.399,"327.516,35",20/04/2022,14:05:04
8,BA.SABADELL,"0,7772","4,94","0,7800","0,7442",20.820.551,"15.985,94",20/04/2022,14:05:13
9,BANKINTER,"5,4140","2,46","5,4480","5,3100",1.220.592,"6.584,03",20/04/2022,14:03:07


In [97]:
df2.to_csv('MarketsSEL.csv')

As a summary for this exercise, we have notice that Selenium it is a bit more "user freindly" since, with the way it connects on a webpage and then (if there aren't any pop-up menu involved) how it navigates from url to url fetching the information we need. Although, that when we need to download the data, we have found that we are required to do the remapping by hand, neither library contains a function that downloads fully function for python the values inside the table-brackets.


# Nivell 2

## Exercici 2: Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.




The description of the dataset can be found in the file [MarketData](https://github.com/isaura-o/ITAcademia/blob/main/Sprint12/Markets_Summary.pdf). For this exercise we only will write down the information of the dataset "MarketsSEL.csv", since it is analogous of doing the same for the first dataset.

Let's clean up the dataset for finding more descriptors for our documentation.

In [116]:
df2 = pd.read_csv('MarketsSEL.csv')

In [117]:
df2.drop('Unnamed: 0', axis=1, inplace=True)

In [118]:
df2.head()

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"196,0000","-0,25","196,6000","192,7000",51.357,"9.985,06",20/04/2022,14:05:01
1,ACERINOX,"10,0700","1,45","10,1500","9,9300",549.250,"5.513,00",20/04/2022,14:03:32
2,ACS,"24,8300","0,36","24,9900","24,8100",195.254,"4.856,16",20/04/2022,14:03:08
3,AENA,"149,7500","-0,27","150,5000","148,5500",37.897,"5.663,36",20/04/2022,14:05:10
4,ALMIRALL,"12,2900","-0,49","12,3600","12,1900",100.272,"1.230,18",20/04/2022,14:04:21


In [119]:
df2.columns

Index(['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen',
       'Efectivo (miles €)', 'Fecha', 'Hora'],
      dtype='object')

In [120]:
# clean up the dataset:
# First we replace commas and points from the columns: Ult, dif, max, min, volumen and, efectivo.
df2['Últ.'] = df2['Últ.'].str.replace(',','.').astype(float)
df2['% Dif.'] = df2['% Dif.'].str.replace(',','.').astype(float)
df2['Máx.'] = df2['Máx.'].str.replace(',','.').astype(float)
df2['Mín.'] = df2['Mín.'].str.replace(',','.').astype(float)
df2['Volumen'] = df2['Volumen'].str.replace('.','').astype(int)
df2['Efectivo (miles €)'] = df2['Efectivo (miles €)'].str.replace('.','').str.replace(',','.').astype(float)

/tmp/ipykernel_3717/350888119.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['Volumen'] = df2['Volumen'].str.replace('.','').astype(int)
/tmp/ipykernel_3717/350888119.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['Efectivo (miles €)'] = df2['Efectivo (miles €)'].str.replace('.','').str.replace(',','.').astype(float)


In [123]:
df2.head()

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,196.00,-0.25,196.60,192.70,51357,9985.06,20/04/2022,14:05:01
1,ACERINOX,10.07,1.45,10.15,9.93,549250,5513.00,20/04/2022,14:03:32
2,ACS,24.83,0.36,24.99,24.81,195254,4856.16,20/04/2022,14:03:08
3,AENA,149.75,-0.27,150.50,148.55,37897,5663.36,20/04/2022,14:05:10
4,ALMIRALL,12.29,-0.49,12.36,12.19,100272,1230.18,20/04/2022,14:04:21


In [121]:
df2.describe(include='all')

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
count,35,35.000000,35.000000,35.000000,35.000000,3.500000e+01,35.000000,35,35
unique,35,NaN,NaN,NaN,NaN,NaN,NaN,1,26
top,ACCIONA,NaN,NaN,NaN,NaN,NaN,NaN,20/04/2022,14:05:02
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,35,3
mean,NaN,28.319720,1.247143,28.444957,27.860220,5.139610e+06,29287.461429,NaN,NaN
std,NaN,40.615752,1.484658,40.771178,40.068732,1.752630e+07,81472.771219,NaN,NaN
min,NaN,0.777200,-1.340000,0.780000,0.744200,2.740700e+04,1230.180000,NaN,NaN
25%,NaN,7.660000,0.205000,7.687500,7.580000,1.812165e+05,4043.505000,NaN,NaN
50%,NaN,16.195000,0.940000,16.350000,16.015000,2.911600e+05,6584.030000,NaN,NaN
75%,NaN,25.395000,2.140000,25.490000,24.900000,1.823342e+06,13536.435000,NaN,NaN


In [122]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Nombre              35 non-null     object 
 1   Últ.                35 non-null     float64
 2   % Dif.              35 non-null     float64
 3   Máx.                35 non-null     float64
 4   Mín.                35 non-null     float64
 5   Volumen             35 non-null     int64  
 6   Efectivo (miles €)  35 non-null     float64
 7   Fecha               35 non-null     object 
 8   Hora                35 non-null     object 
dtypes: float64(5), int64(1), object(3)
memory usage: 2.6+ KB


To know which values were yestarday at closing we use column: %Dif and we divide it by 100 (since it is a percetage) and multiply with the actual value. If the share has decreased we sum, we sustract if the share has increased.

$$\mbox{val share} = \mbox{ultimo value} - \mbox{ultimo value} * \frac{Dif}{100}$$

In [128]:
# acciona share for yestarday at closing:
196 + 196*0.25/100

196.49

In [129]:
# ACERINOX share for yestarday at closing:
10.07 - 10.07*1.45/100

9.923985

# Nivell 3

## Exercici 3: Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 



In this case we will scrape the webpage AO3, in particular of the last updated fanfics that belong to our favorite OTP in the Lord of the Rings/Silmarillion.


In [2]:
import scrapy

In [4]:
url = 'https://archiveofourown.org'
web = requests.get(url).content

# we use scrapy to parse the content to html:
sel = scrapy.Selector(text = web)

In [5]:
sel

<Selector xpath=None data='<html lang="en">\n  <head>\n    <meta c...'>

In [ ]:
# I use the BS library since the if we preview the variable sel, it doesn't show all the webpage on the notebook:
#soup = BeautifulSoup(web, 'html5lib') 
#soup

We are interessed to move to the href: /media/Books%20*a*%20Literature/fandoms which is on a dropdown menu within the menu fandoms.

In [6]:
# we list all the hyperlinks:
links = sel.xpath('//li[@class="dropdown"]//a/@href').extract()
links

['/menu/fandoms',
 '/media',
 '/media/Anime%20*a*%20Manga/fandoms',
 '/media/Books%20*a*%20Literature/fandoms',
 '/media/Cartoons%20*a*%20Comics%20*a*%20Graphic%20Novels/fandoms',
 '/media/Celebrities%20*a*%20Real%20People/fandoms',
 '/media/Movies/fandoms',
 '/media/Music%20*a*%20Bands/fandoms',
 '/media/Other%20Media/fandoms',
 '/media/Theater/fandoms',
 '/media/TV%20Shows/fandoms',
 '/media/Video%20Games/fandoms',
 '/media/Uncategorized%20Fandoms/fandoms',
 '/menu/browse',
 '/works',
 '/bookmarks',
 '/tags',
 '/collections',
 '/menu/search',
 '/works/search',
 '/bookmarks/search',
 '/tags/search',
 '/people/search',
 '/menu/about',
 '/about',
 '/admin_posts',
 '/faq',
 '/wrangling_guidelines',
 '/donate']

From here we can proceed two ways, the first one is to fetch from the fandom we want to search going to the href from "media" (that includes different links as: Manga, Books, etc.) or we can use the search engine inside the webpage named "tags" and from there search our favourites characters or OTPs or so.

Let's do it with the tags search first:

In [7]:
# we notice that the /tags/search it is in the 21st place
url2 = url + links[21]
url2

'https://archiveofourown.org/tags/search'

In [70]:
web2 = requests.get(url2).content
# we use scrapy to parse the content to html:
sel2 = scrapy.Selector(text = web2)
#soup = BeautifulSoup(web2, 'html5lib') # to better visualization ....
#soup

Now, that we are here if we inspect the webpage we will see that we need to fill a form and click on a dropdown menu.

> Note: I have found out in some pages that forms are filled with the code:
> Request(url="https://www.fakeids.com", formdata={'name': 'Joe', 'country': 'UY'})

Instead of using the tag searcher on the webpage and fill the form we will go first to Books and then handpick our favorite OTP, directly from the fanfic list!

Now, let's find the webpages from LOTR and the Silmarillion. Note! We will use only Silmarillion, since our OTP is very popular there, but it is not expensive to extract both.

In [15]:
# find literature link and LOTR and Silmarillion links:
url3 = url + links[3]
web3 = requests.get(url3).content
# we use scrapy to parse the content to html:
sel3 = scrapy.Selector(text = web3)
#soup = BeautifulSoup(web3, 'html5lib') # just visualization of html
#print(soup)
links4 = sel3.xpath('//ul[@class="tags index group"]//a/@href').extract()
#print(links4)
# for LOTR:
r = re.compile(".*Lord%20of%20the%20Rings")
links4b = list(filter(r.match, links4))
print(links4b)
#for Silmarillion:
r = re.compile(".*Silmarillion")
links4a = list(filter(r.match, links4))
print(links4a)

['/tags/The%20Lord%20of%20the%20Rings%20-%20All%20Media%20Types/works', '/tags/The%20Lord%20of%20the%20Rings%20-%20J*d*%20R*d*%20R*d*%20Tolkien/works']
['/tags/The%20Silmarillion%20and%20other%20histories%20of%20Middle-Earth%20-%20J*d*%20R*d*%20R*d*%20Tolkien/works']


Now, we will select our OTP, since we want the list of this pairing.

In [69]:
# selecting OTP!!!
# We only need one of the three urls, because AO3 redirects the works towards the relationships tags. But,
# first we rewrite the 3 urls we want to explore:
url3a = url + links4a[0] + '?page='# Silmarillion
url3b = url + links4b[0] + '?page='#lotr all media
url3c = url + links4b[1] +'?page=' #lotr books only

web3a = requests.get(url3a).content
sel3a = scrapy.Selector(text = web3a)

#soup = BeautifulSoup(web3a, 'html5lib') # just visualization of html
#print(soup)

otp1 = sel3a.xpath('//li[@class="relationships"]//a/@href').extract()
#print(otp1)
# since our OTP is Angbang the tag we want is: Morgoth Bauglir | Melkor/Sauron | Mairon
# which it is the num 4th of the list, right now!
print(otp1[3])

# if we needed to search over different pages we could use:
otp= []
page = 1
max_pages = 2 # usually our OTP is pretty popular! so we do not need scrap many pages to get the relation-url!
while page <= max_pages:
    urlplus = url3a + str(page)
    r = re.compile(".*Morgoth%20Bauglir%20%7C%20Melkor")
    relation = sel3a.xpath('//li[@class="relationships"]//a/@href').extract()
    otp.append(list(filter(r.match, relation)))
    
    page += 1

otp

/tags/Morgoth%20Bauglir%20%7C%20Melkor*s*Sauron%20%7C%20Mairon/works


[['/tags/Morgoth%20Bauglir%20%7C%20Melkor*s*Sauron%20%7C%20Mairon/works',
  '/tags/F%C3%ABanor%20%7C%20Curufinw%C3%AB*s*Morgoth%20Bauglir%20%7C%20Melkor/works',
  '/tags/Morgoth%20Bauglir%20%7C%20Melkor*s*Sauron%20%7C%20Mairon/works'],
 ['/tags/Morgoth%20Bauglir%20%7C%20Melkor*s*Sauron%20%7C%20Mairon/works',
  '/tags/F%C3%ABanor%20%7C%20Curufinw%C3%AB*s*Morgoth%20Bauglir%20%7C%20Melkor/works',
  '/tags/Morgoth%20Bauglir%20%7C%20Melkor*s*Sauron%20%7C%20Mairon/works']]

In [72]:
# now we finally have the webpage we were searching!!!
otpurl = url + otp1[3]
otpurl

'https://archiveofourown.org/tags/Morgoth%20Bauglir%20%7C%20Melkor*s*Sauron%20%7C%20Mairon/works'

Now, if we wanted we could select other tags (freeforms), but we want the list of the lastest, since AO3, gives the list of the latest updated, we select our crawler to pick only a tiny-little selection, of around 5 pages. 

In [342]:
#Angbang crawler: (there are 58 to crawl!)
max_pages = 5

listfics= []
for page in range(1,max_pages):
    
    urlplus = otpurl + '?page=' + str(page)
    web = requests.get(urlplus).content
    sel = scrapy.Selector(text = web)


    title = sel.css("h4[class=heading] a::text").extract()
    author = sel.xpath('//h4[@class="heading"]//a[@rel="author"]/text()').extract()
    # to get only the titles, we need to extract the authors!
    title =[x for x in title if x not in author]
    
    
    hits = sel.xpath('//dd[@class="hits"]/text()').extract()
    kudos = sel.xpath('//dd[@class="kudos"]//a/text()').extract()
    words = sel.xpath('//dd[@class="words"]/text()').extract()
    
    language = sel.xpath('//dd[@class="language"]/text()').extract()
    #we need to extract "\n" from the language list, it is always the two lasts items of the list:
    #r = re.compile("\n")
    language = list(language[:len(language)-2])
    
    url = sel.css("h4[class=heading] a::attr(href)").extract()
    auurl = sel.css("h4[class=heading] a[rel=author]::attr(href)").extract()
    #to remove author url from column:
    url = [x for x in url if x not in auurl]
    
    update = sel.xpath('//p[@class="datetime"]/text()').extract()
    
    row_info = list(zip(title, author, hits, kudos, words, language, update, url))
    
    listfics.append(row_info)

listfics = np.array(listfics)
fics = listfics.reshape(listfics.shape[0]*listfics.shape[1],listfics.shape[2])

# since we are using the function zip, we can convert our dataset into a pandas dataframe:
df = pd.DataFrame(fics, columns=['title', 'author', 'hits', 'kudos', 'words', 'language', 'update', 'url'])

In [346]:
df.head()

,title,author,hits,kudos,words,language,update,url
0,A Guide To Raising Elflings,Niphredilien (Yellow_Feathered_Faerie),9582,421,"43,531",English,22 Apr 2022,/works/30761684
1,And Thou Hast Given Them Blood to Drink,Hinu,960,46,"71,353",English,22 Apr 2022,/works/37798999
2,Tumblr Prompts: A Guide to Raising Elflings,Niphredilien (Yellow_Feathered_Faerie),38,5,"2,449",English,22 Apr 2022,/works/38542471
3,和恶魔讲道理,katherine_the_archer,25,1,"3,387",中文-普通话 國語,21 Apr 2022,/works/38526940
4,Build Up A New Us,Harp_of_Gold,45574,1425,"310,390",English,20 Apr 2022,/works/17806742


In [322]:
df.to_csv('dfDarkLordShipping.csv')

Now we can do statistics and determine  which are the fics losing the time of reading of! 